In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import datetime
import logging
from app.date_formatter import DateFormatter
from app.client.wb import WBClient
from app.client.google_sheets import GoogleSheetsClient
from app.extractors.advert import AdvertFormatter
from app.extractors.card import CardFormatter
from app.extractors.finreport import FinReportFormatter
from app.stat_aggregator import StatAggregator
from app.report_constructor.report_fields import TAG_TO_FIELD_NAME
from app.report_constructor.report_constructor import ReportConstructor
from dotenv import load_dotenv
load_dotenv()

API_TOKEN = os.getenv("API_TOKEN")
AUTH_COOKIES = {
    'wbx-validation-key': os.getenv('WBX_VALIDATION_KEY'),
    'x-supplier-id-external': os.getenv('X_SUPPLIER_ID_EXTERNAL'),
    'WBTokenV3': os.getenv('WB_TOKEN_V3'),
}

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True 
)

# Дату отчета, название документа и листа в нем
REPORT_DATE = datetime.datetime(2025, 3, 20)
SPREADSHEET_NAME = "rnp-experiment"
WORKSHEET_NAME = "Сумка"

wb_client = WBClient(AUTH_COOKIES, API_TOKEN)
gs_client = GoogleSheetsClient()
advFormatter = AdvertFormatter()
cardFormatter = CardFormatter()
finrepFormatter = FinReportFormatter()
statAggregator = StatAggregator()
reportConstructor = ReportConstructor()

In [4]:
# Получение выбранного листа таблицы и nm_id
worksheet = gs_client.get_worksheet(SPREADSHEET_NAME, WORKSHEET_NAME)
worksheet_rows = worksheet.get_all_values()

nm_id = reportConstructor.get_nm_id(worksheet_rows)

In [5]:
# 1. nm_id всех карточек продавца
cards = wb_client.get_cards()

nm_ids = advFormatter.extract_nm_ids_from_cards(cards)

In [9]:
# 2. Внутренний трафик по nm_id
adv_auto, adv_auction = wb_client.get_adverts()

adv_nms = advFormatter.group_nm_ids_by_adverts(adv_auto, adv_auction)
adv_ids = list(adv_nms.keys())
adv_stat = wb_client.get_adverts_stats(adv_ids, REPORT_DATE)

nm_adv_stats = advFormatter.extract_nm_stats_from_advs(adv_stat)
nm_adv_stats

{329981603: {'adv_views': 1606,
  'adv_clicks': 43,
  'adv_atbs': 4,
  'adv_orders': 0,
  'adv_shks': 0,
  'adv_sum': 409.53999999999996,
  'adv_sum_price': 0},
 330495556: {'adv_views': 446,
  'adv_clicks': 4,
  'adv_atbs': 0,
  'adv_orders': 0,
  'adv_shks': 0,
  'adv_sum': 113.72999999999999,
  'adv_sum_price': 0}}

In [7]:
# 3. Общий трафик (карточки), остатки и выкупы по nm_id
cards_stats = wb_client.get_cards_stats(nm_ids, REPORT_DATE)

nm_card_stats = cardFormatter.extract_nm_stats_from_cards(cards_stats)

In [8]:
# 4. Расходы внутри маркетплейса по nm_id
finreps = wb_client.get_finreports(REPORT_DATE)
finrep_ids = finrepFormatter.extract_finreport_ids(finreps)

finrep_records = []
for id in finrep_ids:
    finrep_stat = wb_client.get_finreport_stat_records(id)
    finrep_records.extend(finrep_stat)

nm_finrep_stats = finrepFormatter.extract_nm_stats_from_finrep_records(finrep_records)

In [64]:
# 5. Объединение всех данных по nm_id в финальный отчет
combined_stats = statAggregator.combine_stats(nm_adv_stats, nm_card_stats, nm_finrep_stats)

reports = reportConstructor.generate_rnp_source(combined_stats)

# report = reportConstructor.generate_stats_source_report(combined_stats)
# gs_client.update_sheet(report, "test_sheet", "отчет 2")

In [65]:
report_dot_date = DateFormatter.get_dot_report_date(REPORT_DATE)
tag_row_idxs = reportConstructor.find_tags_row_idxs(worksheet_rows)
date_col_idx = reportConstructor.find_date_col_idx(worksheet_rows, report_dot_date)
nm_report = reports[nm_id]

for row_tag, row_idx in tag_row_idxs:
    if row_tag not in TAG_TO_FIELD_NAME:
        continue
    
    field_name = TAG_TO_FIELD_NAME[row_tag]
    if field_name in nm_report.index:
        cell_coords = reportConstructor.get_cell_coords(row_idx, date_col_idx)
        cell_value = reportConstructor.get_cell_value(nm_report, field_name, row_tag)
        worksheet.update_cell(*cell_coords, cell_value)